In [1]:
import os
#if not os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
#    raise 'stop'
if not os.path.isdir('d:/flagellar/'):
    deps_path = '/kaggle/usr/lib/flg_packages/'
    !pip install --no-index --find-links {deps_path} --requirement {deps_path}/requirements.txt
    !tar xfvz /kaggle/input/ultralytics-for-offline-install/archive.tar.gz
    !pip install --no-index --find-links=./packages ultralytics
    !rm -rf ./packages

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/flagellar/code/core/')
sys.path.append('/kaggle/input/my-flagellar-library/')
import flg_support as fls
import flg_unet
import flg_numerics
import flg_model
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import flg_yolo

fast_mode = False

In [3]:
all_data = fls.load_all_train_data()
np.random.default_rng(seed=0).shuffle(all_data)

# Pick N tomograms with 1 motor and N tomograms with 0 motors as validation set
N=100
n_motors = np.array([len(d.labels) for d in all_data])
inds_zero = np.argwhere(n_motors==0)[:N,0]
inds_one = np.argwhere(n_motors==1)[:N,0]
inds_test = np.concatenate((inds_zero,inds_one))
inds_train = np.setdiff1d(np.arange(len(n_motors)), inds_test)
inds_test.shape, inds_train.shape

train_data = []
for i in inds_train:
    train_data.append(all_data[i])
test_data = []
for i in inds_test:
    test_data.append(all_data[i])
np.random.default_rng(seed=0).shuffle(test_data)
test_data = test_data
if fast_mode:
    train_data = train_data[1:30]
    test_data = test_data[2:4]
len(train_data), len(test_data)

(444, 200)

In [4]:
importlib.reload(flg_yolo)
model = flg_yolo.YOLOModel()
model.seed = 42
model.run_in_parallel = True
if fast_mode: model.n_epochs = 1
model.train(train_data, test_data)
fls.dill_save(fls.temp_dir+ "temp.pickle", model)

Will process approximately 3123 slices for training


Processing training motors:   0%|          | 0/347 [00:00<?, ?it/s]

Will process approximately 900 slices for validation


Processing validation motors:   0%|          | 0/100 [00:00<?, ?it/s]


Processing Summary:
- Train set: 258 tomograms, 347 motors, 3118 slices
- Validation set: 100 tomograms, 100 motors, 900 slices
- Total: 358 tomograms, 447 motors, 4018 slices

Preprocessing Complete:
- Training data: 258 tomograms, 347 motors, 3118 slices
- Validation data: 100 tomograms, 100 motors, 900 slices
- Dataset directory: d:/flagellar/temp//yolo_dataset/
- YAML configuration: d:/flagellar/temp//yolo_dataset/dataset.yaml

Ready for YOLO training!
Starting YOLO training process...
Directory status:
- Train images exists: True
- Val images exists: True
- Train labels exists: True
- Val labels exists: True
Found original dataset.yaml at d:/flagellar/temp//yolo_dataset/dataset.yaml
Fixing YAML paths in d:/flagellar/temp//yolo_dataset/dataset.yaml
Created fixed YAML at d:/flagellar/temp/fixed_dataset.yaml with path: d:/flagellar/temp//yolo_dataset/
Using YAML file: d:/flagellar/temp/fixed_dataset.yaml
YAML contents:
names:
  0: motor
path: d:/flagellar/temp//yolo_dataset/
train: 

...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:07

                   all        900        900      0.915      0.845      0.891      0.344
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 11, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



21 epochs completed in 0.305 hours.
Optimizer stripped from d:\flagellar\temp\yolo_weights\motor_detector\weights\last.pt, 52.0MB
Optimizer stripped from d:\flagellar\temp\yolo_weights\motor_detector\weights\best.pt, 52.0MB

Validating d:\flagellar\temp\yolo_weights\motor_detector\weights\best.pt...
Ultralytics 8.3.98  Python-3.10.14 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs



                Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:11

                   all        900        900      0.694       0.83      0.793       0.36
Speed: 0.1ms preprocess, 7.8ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to d:\flagellar\temp\yolo_weights\motor_detector
MLflow: results logged to runs\mlflow
MLflow: disable with 'yolo settings mlflow=False'

Training complete!


In [ ]:
if not fls.is_submission:
    importlib.reload(flg_yolo)
    model = fls.dill_load(fls.temp_dir + "temp.pickle")
    inferred_test_data = model.infer(test_data)
    print(fls.score_competition_metric(inferred_test_data, test_data))

Using GPU: NVIDIA GeForce RTX 4070 Ti with 12.88 GB memory
Dynamic batch size set to 32 based on 12.84GB free memory
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
Processing tomogram d:/flagellar/data//train/tomo_ff505c (1/1)
Processing 300 out of 300 slices (CONCENTRATION=1)
[PROFILE] Inference batch 1/4: 4.804s
[PROFILE] Inference batch 2/4: 0.164s
[PROFILE] Inference batch 3/4: 0.152s
[PROFILE] Inference batch 4/4: 0.158s
[PROFILE] Inference batch 1/4: 0.175s
[PROFILE] Inference batch 2/4: 0.164s
[PROFILE] Inference batch 3/4: 0.156s
[PROFILE] Inference batch 4/4: 0.121s
[PROFILE] Inference batch 1/4: 0.173s
[PROFILE] Inference batch 2/4: 0.174s
[PROFILE] Inference batch 3/4: 0.148s
[PROFILE] Inference batch 4/4: 0.168s
[PROFILE] Inference batch 1/4: 0.185s
[PROFILE] Inference batch 2/4: 0.173s
[PROFILE] Inference batch 3/4: 0.179s
[PROFILE] Inference batch 4/4: 0.178s
[PROFILE] Inference batch 1/4: 0.196s
[PROFILE] Inference batch 2/4: 0.173s
[PR

In [ ]:
inferred_test_data2 = model.infer(fls.load_all_test_data())
fls.write_submission_file(inferred_test_data2)

In [ ]:
inferred_test_data2